In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import paddle
print("paddle " + paddle.__version__)
device = paddle.set_device("gpu")  # 指定设备
print("using {} device.".format(device))

paddle 2.0.1
using CUDAPlace(0) device.


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:

import zipfile
with zipfile.ZipFile("data/data35095/train.zip") as zf:
   zf.extractall()

import os
from shutil import copy, rmtree
import random


def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)

! mv train data/dataset

# 保证随机可复现
random.seed(0)

# 将数据集中10%的数据划分到验证集中
split_rate = 0.1

# 指向你解压后的flower_photos文件夹
# cwd = os.getcwd()
data_root = "data/"
origin_garbage_path = "data/dataset"
assert os.path.exists(origin_garbage_path)
garbage_class = [cla for cla in os.listdir(origin_garbage_path)
                if os.path.isdir(os.path.join(origin_garbage_path, cla))]

# 建立保存训练集的文件夹
train_root = os.path.join(data_root, "train")
mk_file(train_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(train_root, cla))

# 建立保存验证集的文件夹
val_root = os.path.join(data_root, "val")
mk_file(val_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(val_root, cla))

for cla in garbage_class:
    cla_path = os.path.join(origin_garbage_path, cla)
    images = os.listdir(cla_path)
    num = len(images)
    # 随机采样验证集的索引
    eval_index = random.sample(images, k=int(num*split_rate))
    for index, image in enumerate(images):
        if image in eval_index:
            # 将分配至验证集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(val_root, cla)
            copy(image_path, new_path)
        else:
            # 将分配至训练集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(train_root, cla)
            copy(image_path, new_path)
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    print()

print("processing done!")

[37] processing [312/312]
[18] processing [352/352]
[27] processing [526/526]
[24] processing [308/308]
[34] processing [385/385]
[22] processing [365/365]
[33] processing [312/312]
[35] processing [341/341]
[20] processing [216/216]
[19] processing [302/302]
[25] processing [540/540]
[4] processing [377/377]
[11] processing [726/726]
[36] processing [255/255]
[15] processing [409/409]
[28] processing [372/372]
[39] processing [427/427]
[29] processing [406/406]
[3] processing [75/75]
[14] processing [347/347]
[31] processing [436/436]
[32] processing [270/270]
[30] processing [311/311]
[1] processing [360/360]
[9] processing [379/379]
[16] processing [342/342]
[38] processing [381/381]
[2] processing [269/269]
[10] processing [377/377]
[13] processing [399/399]
[7] processing [352/352]
[23] processing [299/299]
[21] processing [647/647]
[12] processing [321/321]
[6] processing [385/385]
[8] processing [370/370]
[26] processing [341/341]
[5] processing [279/279]
[0] processing [232/232

In [ ]:

import paddle.nn as nn  # 包含了神经网络的主要元素
import paddle

class VGG(nn.Layer):
    def __init__(self, features, num_classes=1000):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),  # 减少过拟合
            nn.Linear(512*7*7, 2048, weight_attr=nn.initializer.KaimingNormal()),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048, weight_attr=nn.initializer.KaimingNormal()),
            nn.ReLU(),
            nn.Linear(2048, num_classes, weight_attr=nn.initializer.KaimingNormal())
        )

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.features(x)
        # N x 512 x 7 x 7
        x = paddle.flatten(x, start_axis=1)  # 展平处理，从第一个维度开始
        # N x 512*7*7
        x = self.classifier(x)
        return x

In [ ]:
def make_features(cfg: list):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2D(kernel_size=2, stride=2)]
        else:
            conv2D = nn.Conv2D(in_channels, v, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal())
            layers += [conv2D, nn.ReLU()]
            in_channels = v
    return nn.Sequential(*layers)
# 产生提取特征网络结构
# 遍历得到列表，结果通过sequential用非关键字参数传入
# 以vgg-16为例：
# 输入是224*224*3，经过一个3*3的卷积核，大小变为（224+2-3）+1=224，channel变为64；再通过一个3*3*64的卷积核仍是这样
# max-pooling层：（224-2）/2+1=112
# 两个卷积层之后channel变为128，feature map变为112*112*128
# maxpooling层之后（112-2）/2+1=56
# 三个卷积层后变为56*56*256
# 以此类推：56*56*256 -> 28*28*256 -> 28*28*512 -> 14*14*512 -> 14*14*512 ->7*7*512
# 最后进入全连接层

In [ ]:
cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}
# 四种不同结构，数字代表卷积层中数字个数，M代表池化层

def vgg(model_name="vgg16", **kwargs):  # 实例化
    try:
        cfg = cfgs[model_name]
    except:
        print("Warning: model number {} not in cfgs dict!".format(model_name))
        exit(-1)
    model = VGG(make_features(cfg), **kwargs)  # 传入features
    # 后面的是字典变量
    return model

In [ ]:
import os
import json
import time

import paddle
import paddle.nn as nn

from paddle.vision import transforms
from paddle.vision import datasets
import paddle.optimizer as optim
from paddle.static import InputSpec

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

data_transform = {
    "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    "val": transforms.Compose([transforms.Resize((224, 224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}
# 预处理

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-03-30 13:11:15,278 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

In [ ]:
train_dataset = datasets.DatasetFolder(root="data/train",
                                     transform=data_transform["train"])
train_num = len(train_dataset)

garbage_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in garbage_list.items())  # key和val反一下

# write dict into json file

json_str = json.dumps(cla_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)

batch_size = 128
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=batch_size, shuffle=True,
                                    num_workers=0)

# windows只能为0，线程数(num_workers)

validate_dataset = datasets.DatasetFolder(root="data/val",
                                          transform=data_transform["val"])
val_num = len(validate_dataset)
validate_loader = paddle.io.DataLoader(validate_dataset,
                                        batch_size=batch_size, shuffle=True,
                                        num_workers=0)
print("using {} images for training, {} images for validation.".format(train_num,
                                                                       val_num))

using 12980 images for training, 1422 images for validation.


In [13]:
model_name = "vgg16"
net = vgg(model_name=model_name, num_classes=40)  # 进行实例化
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters=net.parameters(), learning_rate=0.0001)

epochs = 200
best_acc = 0.0
# save_path = './{}Net.pth'.format(model_name)
train_steps = len(train_loader)
for epoch in range(epochs):
    # train
    net.train()
    running_loss = 0.0
    t1 = time.perf_counter()
    # train_bar = tqdm(train_loader)
    for step, data in enumerate(train_loader, start=0):
        images, labels = data
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad() # 梯度置零

        # print statistics
        running_loss += loss.numpy()[0]
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        loss_num = loss.numpy()[0]
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss_num), end=" ")
    print()
    print(time.perf_counter()-t1)

    # validate
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    with paddle.no_grad():
        for val_data in validate_loader:
            val_images, val_labels = val_data
            outputs = net(val_images)
            predict_y = outputs.numpy().argmax(axis=1)
            acc += (val_labels.astype("int").numpy()==outputs.numpy().argmax(axis=1).astype("int")).sum()

    val_accurate = acc / val_num
    print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
          (epoch + 1, running_loss / train_steps, val_accurate))


    if val_accurate > best_acc:
        best_acc = val_accurate
        paddle.jit.save(
        layer=net,
        path="model",
        input_spec=[InputSpec(shape=[1, 3, 224, 224], dtype='float32')])
        

print('Finished Training')

train loss:  1 %[->.................................................]3.961 

KeyboardInterrupt: 

In [ ]:
!mv data/data77426/model.pdiparams model.pdiparams 

In [ ]:
import zipfile
with zipfile.ZipFile("data/data35095/test.zip") as zf:
   zf.extractall()

data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
with open("test.txt","w") as f:
    for i in range(1,401):
        img_path = "test/test"+str(i)+".jpg"
        assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
        img = Image.open(img_path)

        paddle.disable_static()
        # plt.imshow(img)
        # [N, C, H, W]
        img = data_transform(img)
        # expand batch dimension
        img = paddle.unsqueeze(img, axis=0)

        # read class_indict
        json_path = 'class_indices.json'
        assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

        json_file = open(json_path, "r")
        class_indict = json.load(json_file)

        # create model
        model = vgg(model_name='vgg16', num_classes=40)

        # load model weights
        # weights_path = "AlexNet.pth"
        # assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
        model = paddle.jit.load("model")
        # model.load_state_dict(torch.load(weights_path))

        model.eval()
        with paddle.no_grad():
            # predict class
            output = paddle.squeeze(model(img))
            predict = nn.Softmax(axis=0)(output)
            predict_cla = paddle.argmax(predict).numpy()[0]

        # print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                    #predict.numpy()[predict_cla])
        # print(print_res)
    
        f.write(class_indict[str(predict_cla)]+"\n")

        rateTest = i / 400
        aTest = "*" * int(rateTest * 50)
        bTest = "." * int((1 - rateTest) * 50)
        print("\rprocess: {:d}% [{}->{}]".format(int(rateTest * 100), aTest, bTest), end=" ")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/backward.py:1640: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return list(x) if isinstance(x, collections.Sequence) else [x]


process: 23% [***********->......................................]  